In [1]:
import glob
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import csv
import json
!pwd

/data/projects/STUDIES/social_doors_college/code


In [2]:
!ls ../

behavior  bids	code  derivatives  fmriprep.sh	outdated  re-runL1.log


In [3]:
def event_writer(event_type, df_series, duration, block_type):
    '''
    Funciton takes a string, pandas series, and a float/int
    This function 
    1. Converts ms in series to s, and makes dataframe obj
    2. Duration is appended as a 'default value' column
    3. Makes column of ones (expected by fsl) and adds to df
    4. Writes out to csv with index and header turned off
    '''
    df = (df_series/1000).to_frame()
    df['duration'] = duration
    df['mag'] = 1.0
    df.to_csv(f'../behavior/EVfiles/{sub}/{condition_type}/run-0{run}_{block_type}_{event_type}.txt', index = False, header = False, sep = '\t')
    

def extract_fd_vals(json_file):
    f = open(json_file)
    data = json.load(f)
    fd_data = data['fd_mean'] 
    f.close() 
    return fd_data

### Loop Over Each Social/Doors excel file (raw data)

In [4]:
#make a list for logging an intermediate file

interm = {}

'''
Split excel files into 4, convert to 3 'colmumn file' within sub folders

iterate over each excel file - 2 per person: social, nonsocial

2 blocks within each run

4 EVs in question: instruction, correct, incorect and decision

'''

for excel in glob.glob('../behavior/raw/s*/*/*.xlsx')[:]:
    
    #get sub number from filename
    sub = excel[17:21]
    print(sub)
    #load into pandas dataframe 
    df = pd.read_excel(excel, header = 1)
    

    #define block list
    split_df = np.array_split(df, 4)
    
    
    #iterate over blocks in excel file
    for block in split_df:
        '''
        get the columns for scanner offset, decision onsets, and feedback onsets 
        This uses the index of columns, which switches between the social and non-social files
        
        ''' 
        condition_type = 'social' if 'Social' in df.iloc[0][0] else 'doors'
        
        
        block_type = block['Procedure[Trial]'].iloc[0]
        
        block_type = 'positive' if ('Prize' in block_type) or ('Like' in block_type) else 'negative'
        
        
        
        if condition_type == 'social':
            offset_value = block.iloc[1, 56]
            decision_onsets = block.iloc[:, 82].dropna()  #get rid of nans  when possible
            
            
            
            
        
        else: #nonsocial
            offset_value = block.iloc[1, 40]
            decision_onsets = block.iloc[:, 51].dropna()
            
            
            
            
            
        #get decision onsets where R or L
        
        
        #print(decision_onsets, responses)
         
        
        all_cols = block.columns.tolist()
        for i in all_cols:
            if '.RESP' in i and ('Faces' in i or 'Doors' in i):
                response_column = i
            else:
                pass
        
        
        response_b = block.loc[block[response_column].eq('b')]
        response_g = block.loc[block[response_column].eq('g')]
        
        
    
        
        
        if condition_type == 'social':
            response_b_onsets = response_b.iloc[:, 82].dropna()
            response_g_onsets = response_g.iloc[:, 82].dropna()#get rid of nans  when possible
        else: #nonsocial
            response_b_onsets = response_b.iloc[:, 51].dropna()
            response_g_onsets = response_g.iloc[:, 51].dropna()
        
        #alternate,cleaner method for above
        #offset_value = block.filter(regex='ScannerTrigger*..OffsetTime', axis = 1).iloc[0].iloc[0]
        
        #get feedback onsets for correct and incorrect
        correct_onsets = block.loc[block['TrialType'] =='win', 'Outcome.OnsetTime']
        incorrect_onsets = block.loc[block['TrialType'] =='loss', 'Outcome.OnsetTime']
        
        #assign run based on column named "Block"
        run = block['Block'].iloc[0] 
        
        #assign a or b on column named "Block Number" 
        if block['BlockNumber'].iloc[0] == 1 :
            block_a_b = 'a'
        elif block['BlockNumber'].iloc[0] == 2:
            block_a_b = 'b' 
        else:
            print("Error")       
        
        
        
        #add fdmean values to log 
        
        
        
        
        
        
        #instruction type, as a string, is determined within block 
        instruction_type = block['Procedure[Trial]'].iloc[0][:-4] + 's.OnsetTime'
         
        #make scanner trigger adjustements
        
        adjusted_decision_onsets = decision_onsets.subtract(offset_value)
        
        adjusted_response_b_onsets = response_b_onsets.subtract(offset_value)
        adjusted_response_g_onsets = response_g_onsets.subtract(offset_value)
        
        adjusted_correct_onsets = correct_onsets.subtract(offset_value)
        adjusted_incorrect_onsets = incorrect_onsets.subtract(offset_value)
           
        #use first and last events to get experiment time 
        unadj_start = block[instruction_type].iloc[0]
        unadj_end = block['ITI.OffsetTime'].iloc[-1]
        
        block_start = (unadj_start - offset_value)
        block_end = (unadj_end - offset_value)
        dur = unadj_end - unadj_start
           
        #TR is 2.1 s
        trs = dur/2100
        
            
        '''
        B trials need to be given a new 'zero point'
        '''
        
        #print(block_a_b, adjusted_response_b_onsets)
        
        #reset b runs to block onsets, additional TR correction 
        if block_a_b == 'b':
            #print(True)
            #check for negatives
            #data correspond to which part of run
            
            #rename variables here, double check values
            
            adjusted_decision_onsets = adjusted_decision_onsets.subtract(block_start)
            
            adjusted_response_b_onsets = adjusted_response_b_onsets.subtract(block_start)
            adjusted_response_g_onsets = adjusted_response_g_onsets.subtract(block_start)
            
            adjusted_correct_onsets = adjusted_correct_onsets.subtract(block_start)
            adjusted_incorrect_onsets = adjusted_incorrect_onsets.subtract(block_start)
            
        else:
            pass
    
        #print(adjusted_response_b_onsets)
        
        instruct_list = [0., 5., 1.]
        
        #write EVs to file
        #event_writer('correct', adjusted_correct_onsets, 1., condition_type)
        #event_writer('incorrect', adjusted_incorrect_onsets, 1., condition_type)
        #event_writer('decision', adjusted_decision_onsets, 3., condition_type)
        
        event_writer('Rdecision', adjusted_response_b_onsets, 3., block_a_b)
        event_writer('Ldecision', adjusted_response_g_onsets, 3., block_a_b)
        
        
        
        
        #with open(f'../behavior/EVfiles/{sub}/{condition_type}/run-0{run}_{block_type}_instruction.txt', 'w', newline='') as csvfile:
        #        writer = csv.writer(csvfile, delimiter='\t',
        #                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
        #        writer.writerow(instruct_list)
    
        #slicing the block out of the NIFTI  file
        
        if trs > 66:
            block_length = 'long'
            stop = 66
            if block_a_b == 'a':
                start = 0
            elif block_a_b == 'b' and block_length == 'long':
                start = 64
        elif trs < 65:
            block_length = 'short'
            stop = 64
            if block_a_b == 'a':
                start = 0
            elif block_a_b == 'b' and block_length == 'short':
                start = 66
    

        d1 = {sub + f'_{condition_type}_'+'_run_' + str(run) + '_block_' + block_a_b: [start, stop, block_length, block_a_b, condition_type, run, trs, block_type]}
        interm.update(d1)

        #if condition_type == 'social':
        #    infile = f'../bids/sub-{sub}/func/sub-{sub}_task-srSocial_run-0{run}_bold.nii.gz' 
        #    outfile = f'../bids/sub-{sub}/func/sub-{sub}_task-srSocial{block_a_b}_run-0{run}_bold.nii.gz'
        #    !fslroi $infile $outfile $start $stop 
        #else:
        #    infile = f'../bids/sub-{sub}/func/sub-{sub}_task-srDoors_run-0{run}_bold.nii.gz' 
        #    outfile = f'../bids/sub-{sub}/func/sub-{sub}_task-srDoors{block_a_b}_run-0{run}_bold.nii.gz'
        #    !fslroi $infile $outfile $start $stop 
    

3847
3847
3865
3865
3910
3910
3849
3849
3864
3864
3852
3852
4017
4017
3891
3891
3886
3886
3846
3846
4020
4020
4019
4019
3877
3877
3889
3889
3854
3854
3887
3887
4018
4018
3882
3882
3880
3851
3851
3855
3855
3992
3992
3967
3967
3895
3895
3871
3871
3892
3892
3883
3883
3920
3920
3848
3848
3893
3893
3845
3845
3912
3912
3836
3836
3890
3890


### Writing out a logging csv for test

In [5]:
df = pd.DataFrame(interm).transpose()
df.columns = ['Starting Volume', 'Length in Volumes', 'Long or Short', 'Block Type', 'Condition', 'Run Number', 'N Volumes', 'Instruction Valence']
df.to_csv('test_output.csv')


# Run First Level Stats

In [6]:
import glob
from csv import reader


for i in glob.glob('../behavior/EVfiles/*'):
    sub_number = i[20:24]
    print(sub_number)
    with open('test_output.csv', 'r') as read_obj:
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        for row in csv_reader:
            if sub_number == row[0][:4]:
                trial_type = row[0][-1:]
                nvols = row[2]
                condition = row[0][5:11]
                condition = condition.replace("_", "").capitalize()
                print(trial_type, nvols, condition)
                !./models/L1_task-sr$condition$trial_type\_\model-01.sh $sub_number 2 0 6 $nvols
                !./models/L1_task-sr$condition$trial_type\_\model-01.sh $sub_number 1 0 6 $nvols
        read_obj.close()

3877
a 64 Doors
/data/projects/STUDIES/social_doors_college
/data/projects/STUDIES/social_doors_college
b 66 Doors
a 64 Doors
/data/projects/STUDIES/social_doors_college
/data/projects/STUDIES/social_doors_college
b 66 Doors
a 66 Social
b 64 Social
a 66 Social
b 64 Social
4020
a 64 Doors
/data/projects/STUDIES/social_doors_college
/data/projects/STUDIES/social_doors_college
b 66 Doors
a 64 Doors
/data/projects/STUDIES/social_doors_college
/data/projects/STUDIES/social_doors_college
b 66 Doors
a 66 Social
b 64 Social
a 66 Social
b 64 Social
3967
a 66 Doors
/data/projects/STUDIES/social_doors_college
/data/projects/STUDIES/social_doors_college
b 64 Doors
a 66 Doors
/data/projects/STUDIES/social_doors_college
/data/projects/STUDIES/social_doors_college
b 64 Doors
a 66 Social
b 64 Social
a 66 Social
b 64 Social
3882
a 66 Doors
/data/projects/STUDIES/social_doors_college
/data/projects/STUDIES/social_doors_college
b 64 Doors
child process exited abnormally
    while executing
"fsl:exec "${F

/data/projects/STUDIES/social_doors_college
child process exited abnormally
    while executing
"fsl:exec "${FSLDIR}/bin/feat_model design" $fmri(featModelOpts) -N feat0_model -l logs "
    (procedure "firstLevelMaster" line 43)
    invoked from within
"firstLevelMaster $session"
    invoked from within
"if { $done_something == 0 } {

    if { ! $fmri(inmelodic) } {
       if { $fmri(level) == 1 } {
	      for { set session 1 } { $session <= $fmri(mult..."
    (file "/usr/share/fsl/5.0/bin/feat" line 387)
b 64 Doors
child process exited abnormally
    while executing
"fsl:exec "${FSLDIR}/bin/feat_model design" $fmri(featModelOpts) -N feat0_model -l logs "
    (procedure "firstLevelMaster" line 43)
    invoked from within
"firstLevelMaster $session"
    invoked from within
"if { $done_something == 0 } {

    if { ! $fmri(inmelodic) } {
       if { $fmri(level) == 1 } {
	      for { set session 1 } { $session <= $fmri(mult..."
    (file "/usr/share/fsl/5.0/bin/feat" line 387)
a 64 Social